In [1]:
from cpptraj_Analysis_Frames import *

In [25]:
class Application(tk.Frame):
    def __init__(self, master=None):
        super().__init__(master)
        ### INTERNAL VARIABLES ###
        self.BooleanOptions = ["Autoimage","StripWater","StripIons"]
        self.master = master
        self.pack(fill="both",expand=True)

        ### FRAMES ###
        ## TEXT SIDE
        self.TEXT = tk.Frame(self)
        self.TEXT.pack(side="right",fill="both",expand=True)
        self.display_commands_frame = tk.Frame(self.TEXT)
        self.frame_run_or_quit = tk.Frame(self.TEXT)
        ## NON TEXT SIDE
        self.NONTEXT = tk.Frame(self)
        self.NONTEXT.pack(side="left",fill="both",expand=True)

        ### FINAL INITIALIZATION OF PROGRAM ###
        self.pack()
        self.create_widgets()

    def create_widgets(self):
        ### FileFrames
        self.PRMTOP = FILE_SELECT_BUTTON(self.NONTEXT,"Prmtop File",(("AMBER Parm7","*.prmtop*"),("all files","*.*")))
        self.TRAJECTORY  = FILE_SELECT_BUTTON(self.NONTEXT,"Trajectory File",(("CHARMM DCD","*.dcd*"),("AMBER MDCRD","*.mdcrd*"),("all files","*.*")))

        ### RadioButton Options
        self.AUTOIMAGE = BOOLEAN_WITH_ENTRY(self.NONTEXT,"Autoimage?","Selection Mask",":1-100","","autoimage anchor ","")
        self.STRIPWATER = BOOLEAN_WITH_ENTRY(self.NONTEXT,"Strip solvent?","Selection Mask",":WAT","","strip ","")
        self.STRIPIONS = BOOLEAN_WITH_ENTRY(self.NONTEXT,"Strip counterions?","Selection Mask",":K+,CL-","","strip ","")
        
        # Add Analyses
        self.ANALYSES = Analysis_Frame(self.NONTEXT)
        self.ANALYSES.pack(fill="both")
        
        # DISPLAY CURRENT COMMANDS
        self.display_commands_frame.pack(side="top",fill="both")
        self.T = tk.Text(self.display_commands_frame, height=40, width=120)
        self.commands_list="this is a test of commands\nsecondline\n now a third line\n  four score and seven whatever"
        self.T.pack()
        
        # RUN CPPTRAJ or QUIT Frame
        self.frame_run_or_quit.pack(side="bottom",fill="both")
        self.quit = tk.Button(self.frame_run_or_quit,text="QUIT",fg="red",command=root.destroy)
        self.quit.pack(side="right")
        self.run_cpptraj = tk.Button(self.frame_run_or_quit,text="RUN CPPTRAJ",fg="green",command=self.run_cpptraj)
        self.run_cpptraj.pack(side="right")
        self.update_text = tk.Button(self.frame_run_or_quit,text="UPDATE_TEXT",fg="blue",command=self.update_Text)
        self.update_text.pack(side="right")
        
        
    def run_cpptraj(self):
        self.update_Text()
        with open("cpptraj.in","w") as f:
            f.write(self.commands_list)
        f.close()
        subprocess.call("cpptraj < cpptraj.in > cpptraj.out", shell=True)
        # generate cpptraj.in based on all variables, then call cpptraj.

    
    def update_Text(self):
        self.commands_list = commentary("")
        self.commands_list += f"parm {self.PRMTOP.value()}\n"
        self.commands_list += f"trajin {self.TRAJECTORY.value()}\n"
        self.commands_list += f"{self.AUTOIMAGE.value()}\n"
        self.commands_list += f"{self.STRIPWATER.value()}\n"
        self.commands_list += f"{self.STRIPIONS.value()}\n"
        self.commands_list += self.ANALYSES.command_string()
        self.commands_list += "run\n"
        
        ### UPDATE DISPLAYED TEXT ###    
        self.T.delete(1.0,tk.END)
        self.T.insert(tk.END,self.commands_list) 
        self.T.config(height=max(len(self.commands_list.split("\n"))+1,40))
        self.T.pack()








In [26]:
def _on_mousewheel(self, event):
    self.canvas.yview_scroll(-1*(event.delta/120), "units")


### RUN MAIN APPLICATION
root = tk.Tk()
root.title("CPPTRAJ Analysis Generator")
root.geometry("1150x500")

#Scrollable Main Window
main_frame = tk.Frame(root)
main_frame.pack(fill="both",expand=True)
scroll = tk.Frame(main_frame)
scroll.pack(fill="y",side="right")
main_canvas= tk.Canvas(main_frame)
main_canvas.pack(side="left",fill="both",expand=True)
y_scrollbar = tk.Scrollbar(main_frame,orient="vertical",command=main_canvas.yview)
y_scrollbar.pack(side="right",fill="y")
main_canvas.configure(yscrollcommand=y_scrollbar.set)
root.bind("<Configure>",lambda e: main_canvas.config(scrollregion= main_canvas.bbox("all"))) 
root.bind("<FocusOut>",lambda e: main_canvas.config(scrollregion= main_canvas.bbox("all"))) 
root.bind("<FocusIn>",lambda e: main_canvas.config(scrollregion= main_canvas.bbox("all"))) 
root.bind_all("<MouseWheel>", _on_mousewheel)



submain = tk.Frame(main_canvas)
main_canvas.create_window((0,0), window=submain, anchor="nw")




app = Application(submain)


root.mainloop()